<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/PunctFormer/blob/main/notebooks/seq2seq_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Approaches:

1. LSTM for next word(here punctuations) predictions
2. Next word(punctuation) prediction using Transformer
3. **Simple Seq2Seq model using Transformers**
4. Smaller model from scratch pre-train and then finetune for punctuation restoration

[https://discuss.huggingface.co/t/how-to-train-a-translation-model-from-scratch/160/7](https://discuss.huggingface.co/t/how-to-train-a-translation-model-from-scratch/160/7)


In [ ]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp36-cp36m-linux_x86_64.whl

In [ ]:
!git clone https://github.com/huggingface/transformers.git
!pip install -e /content/transformers

!pip install wandb

In [ ]:
!pip install -r "/content/transformers/examples/requirements.txt"
!pip install --upgrade pyarrow

In [ ]:
!pip install --upgrade  transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Restart runtime

In [ ]:
import gc
import logging

import torch

import numpy as np
import pandas as pd

## Data collectiona and separation

different files for training data

In [ ]:
!wget http://hltshare.fbk.eu/IWSLT2012/IWSLT12.TALK.train.en

In [ ]:
!wget http://hltshare.fbk.eu/IWSLT2012/training-monolingual-europarl.tgz

In [ ]:
!wget https://www.statmt.org/europarl/v7/fr-en.tgz

In [ ]:
!tar -xvf /content/fr-en.tgz

In [ ]:
with open("/content/training-monolingual-europarl/europarl-v7.en") as fp:
    lines = fp.readlines()

In [ ]:
#lines = lines[:int(len(lines) * 0.1)]

In [ ]:
import string

In [ ]:
punctuations = "?!,-:;."

In [ ]:
def cleanString(strval):
    return "".join(" " if i in punctuations else i for i in strval.lower().strip(punctuations))

In [ ]:
dest = []
for i, line in enumerate(lines):
    lines[i] = line.replace("\n", "")
    s = " ".join(cleanString(i) for i in line.split())
    dest.append(s)

In [ ]:
len(lines), len(dest)

In [ ]:
lines[:100]

In [ ]:
dest[:100]

In [ ]:
df = pd.DataFrame({"input_text":dest, "target_text":lines})

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df

In [ ]:
train_data, eval_data = train_test_split(df, test_size=0.3)
eval_data, test_data = train_test_split(eval_data, test_size=0.5)

train_df = train_data
eval_df = eval_data
test_df = test_data

In [ ]:
train_df.isna().sum()

In [ ]:
eval_df.isna().sum()

---

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
train_df

---

In [ ]:
!cp -r /content/outputs/best_model /content/gdrive/MyDrive/DAL/punctuation/bart

In [ ]:
gc.collect()

# TPU - Europarlv7  with ?!,-:;.

In [ ]:
train_df["input_text"].apply(lambda x:len(x.split(" "))).sum()

In [ ]:
for ix in train_df.index:
    
    src_txt = train_df["input_text"][ix]
    if src_txt=="":
        print(ix)

In [ ]:
train_df["target_text"].values[:100]

data needs to be in the files

- train.source
- train.target
- val.source
- val.target
- test.source
- test.target

In [ ]:
with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/train.source", 'w') as fp_src:
    with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/train.target", 'w') as fp_tgt:
        for ix in train_df.index:
            src_txt = train_df["input_text"][ix]
            dest_txt = train_df["target_text"][ix]
            #skipping empty strings
            if src_txt != "":
                fp_src.write("{}\n".format(src_txt))
                fp_tgt.write("{}\n".format(dest_txt))

gc.collect()

with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/val.source", 'w') as fp_src:
    with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/val.target", 'w') as fp_tgt:
        for ix in eval_df.index:
            src_txt = eval_df["input_text"][ix]
            dest_txt = eval_df["target_text"][ix]
            #skipping empty strings
            if src_txt != "":
                fp_src.write("{}\n".format(src_txt))
                fp_tgt.write("{}\n".format(dest_txt))

gc.collect()

with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/test.source", 'w') as fp_src:
    with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/test.target", 'w') as fp_tgt:
        for ix in test_df.index:
            src_txt = test_df["input_text"][ix]
            dest_txt = test_df["target_text"][ix]
            #skipping empty strings
            if src_txt != "":
                fp_src.write("{}\n".format(src_txt))
                fp_tgt.write("{}\n".format(dest_txt))

gc.collect()

In [ ]:
with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/train.source", 'r') as fp_src:
    with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/train.target", 'r') as fp_tgt:
        print("train: ", fp_src.read(1000), "\n\nop:", fp_tgt.read(1000))

gc.collect()

with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/val.source", 'r') as fp_src:
    with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/val.target", 'r') as fp_tgt:
        print("\n\n\n\neval: ", fp_src.read(1000), "\n\nop:", fp_tgt.read(1000))


gc.collect()

In [ ]:
!python "/content/transformers/examples/seq2seq/finetune.py" --help

In [ ]:
gc.collect()

initial checkpoints from a pretrained hosted model

In [ ]:
#set --output_dir to drive to avoid data loss in case of
#runtime disconnect
!python "/content/transformers/examples/seq2seq/finetune_trainer.py" \
    --tpu_cores 1\
    --do_train \
    --train_batch_size 32 \
    --eval_batch_size 32\
    --num_train_epochs 1\
    --max_source_length 32\
    --max_target_length 32\
    --seed 108 \
    --n_val 1000 \
    --save_top_k 3\
    --val_check_interval 0.1 \
    --data_dir "/content/gdrive/MyDrive/DAL/punctuation/data" \
    --model_name_or_path "bart-base" \
    --output_dir "/content/op"\
    #--fp16

In [ ]:
gc.collect()

In [ ]:
with open("/content/gdrive/MyDrive/DAL/punctuation/data/train.source", "r") as fp:
    lines = fp.readlines()

for i, line in enumerate(lines):
    if line=="\n":
        print(i)

In [ ]:
with open("/content/gdrive/MyDrive/DAL/punctuation/data/train.target", "r") as fp:
    lines = fp.readlines()

for i, line in enumerate(lines):
    if line=="\n":
        print(i)

Training on TPU

1 epochs
1. max src-tgt lengths: 64 & device batch_size =128



In [ ]:
gc.collect()

In [ ]:
!cp -r "/content/gdrive/MyDrive/DAL/punctuation/bart/europarl7" "/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap"

from our checkpoint 

In [ ]:
!python "/content/transformers/examples/xla_spawn.py" --num_cores 1\
    "/content/transformers/examples/seq2seq/finetune_trainer.py" \
        --do_train \
        --num_train_epochs 1 \
        --max_source_length 64 \
        --max_target_length 64 \
        --per_device_train_batch_size 128 \
        --per_device_eval_batch_size 128 \
        --n_val 1000\
        --do_eval \
        --seed 108 \
        --task translation \
        --data_dir "/content/gdrive/MyDrive/DAL/punctuation/data_europarl7" \
        --model_name_or_path "/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap" \
        --output_dir "/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap"\
        --overwrite_output_dir \
        --prediction_loss_only \
        #--save_top_k 3\
        #--fp16

# Evaluation

Change runtime to GPU for faster evaluation and testing 

In [ ]:
!python "/content/transformers/examples/seq2seq/finetune_trainer.py" \
        --do_predict \
        --seed 108 \
        --task translation \
        --data_dir "/content/gdrive/MyDrive/DAL/punctuation/data_europarl7" \
        --model_name_or_path "/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap" \
        --output_dir "/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap"\
        --overwrite_output_dir \
        --max_source_length 64 \
        --per_device_eval_batch_size 1\
        #--save_top_k 3\
        #--fp16

In [ ]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base", max_length=400)

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
restore = pipeline('translation_xx_to_yy', model=model.to("cuda"), tokenizer=tokenizer)

In [ ]:
model = BartForConditionalGeneration.from_pretrained("/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap", max_length=50000)

tokenizer = AutoTokenizer.from_pretrained("/content/gdrive/MyDrive/DAL/punctuation/bart/europarl_cap")
restore = pipeline('translation_xx_to_yy', model=model.to("cuda"), tokenizer=tokenizer, device=0)

In [ ]:
%%time
restore("")

CPU times: user 44.1 ms, sys: 3.17 ms, total: 47.3 ms
Wall time: 48.8 ms


[{'translation_text': ''}]

In [ ]:
model.config

In [ ]:
import numpy as np

In [ ]:
with open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/test.target")as tgt_f, open("/content/gdrive/MyDrive/DAL/punctuation/data_europarl7/test.source") as src_f:
    src_lines = src_f.readlines()[:1_000]
    dest_lines = tgt_f.readlines()[:1_000]


In [ ]:
for i, line in enumerate(src_lines):
    src_lines[i] = line.replace("\n", "")

for i, line in enumerate(dest_lines):
    dest_lines[i] = line.replace("\n", "")

In [ ]:
test_df = pd.DataFrame({"input_text":src_lines, "target_text":dest_lines})

In [ ]:
test_df["input_text"].values[0]

'these are only a few examples to show just how important the five year programme is'

In [ ]:
%%time
idx = np.random.randint(0, len(test_df), (1000))
test_df["pred"] = ""
#for ix in idx:
for ix in range(len(test_df)):
    
    print(ix)
    inp_txt = test_df["input_text"].values[ix]
    correct = test_df["target_text"].values[ix]
    punc_txt = restore(inp_txt)[0]["translation_text"]
    test_df.loc[ix, "pred"] = punc_txt

    print(f"Inp: {inp_txt}\n OG: {correct} \nOut: {punc_txt}\n")

In [ ]:
test_df.to_csv("preds.csv", index=False)

In [ ]:
test_df.to_csv("")

In [ ]:
test_df.shape

In [ ]:
max(test_df["target_text"].apply(len))

In [ ]:
from tqdm.auto import tqdm

In [ ]:
%%time

for ix in tqdm(range(len(test_df))):

    inp_txt = test_df["input_text"].values[ix]
    correct = test_df["target_text"].values[ix]
    punc_txt = restore(inp_txt)[0]["translation_text"]
    test_df.loc[ix, "pred"] = punc_txt

'Such a solution would make a better contribution to the task of developing the importance of fisheries for worldwide food supplies, boost these nations&#x02BC; economic development and also remove the risks that may be involved in the fisheries agreements, e.g. that certain Member States&#x02BC; fisheries activities might receive indirect support to continue operating in a situation where the entire European fishing industry is in need of substantial restrictions.\n'